<div class="jumbotron" style="color:#F9F9FB;background-color:#1a242d !important;">
  <span style="float: right;width:5%;height:5%">
      <img src="https://ipfs.io/ipfs/QmVfFbASG11MjUFuvfAJmjKpxRGYstCAw9GmZEpbA7KE7A"/>
  </span>
  <span style="float: right;width:5%;height:5%">
      <img style="background-color:white" src="https://rawgit.com/jupyter/design/master/logo/svg/jupyter-sq-text.svg" />
  </span>
  <h1 class="display-3">InterPlanetary File System (IPFS) on Jupyter</h1>
  <p class="lead">Jovians and Bluemix users rejoice!</p>
  <hr class="m-y-2">
  <p><span style="color:orange;">Art of the Possible</span> sometimes means mashing up Alpha technology with production level hosted cloud platforms ;-)<br/>
  Brought to you courtesy of the IBM jStart Team (<a href="http://ibm.com/jstart" target="_new">http://ibm.com/jstart</a>)</p>
  <p class="lead">
      <div class="text-center">
      <img style="width:10%;height:10%;display:inline" src="https://ipfs.io/ipfs/QmTgtbb4LckHaXh1YhpNcBu48cFY8zgT1Lh49q7q7ksf3M/raster-generated/ipfs-logo-text-128-white.png">&nbsp;&nbsp;
      <a class="btn btn-primary btn-lg" target="_new" href="https://ipfs.io" role="button">Learn more</a></div>
  </p>
</div>

<div class="alert alert-success" role="alert">
  <strong>Let's get started!</strong> We need to install the IPFS Go-based binary into our notebook.<br/>We'll do this by loading a convenient external script and re-running the cell.<br/>
  <strong style="color:purple">Extra Credit</strong>: Check out <a href="https://github.com/ipfs/faq/issues/76" target="_new">how IPFS got its name</a>
</div>

In [ ]:
#  Source: https://raw.githubusercontent.com/joshisa/jspark/master/jspark/ipfs.py

# Useful Reading
#   -   https://ipfs.io/ipfs/QmNhFJjGcMPqpuYfxL62VVB9528NXqDNMFXiqN5bgFYiZ1/its-time-for-the-permanent-web.html
#   -   https://netninja.com/2015/09/30/ipfs_interplanetary_file_system/

# Purpose:  Installation of IPFS (InterPlanetary File System) within Jupyter
#           IPFS is a peer-to-peer distributed file system that seeks to 
#           connect all computing devices with the same system of files.
#           In some ways, IPFS is similar to the World Wide Web, but IPFS 
#           could be seen as a single BitTorrent swarm, exchanging objects 
#           within one Git repository. IPFS has no single point of failure,
#           and nodes do not need to trust each other.
#           The filesystem can be accessed in a variety of ways, including 
#           via FUSE and over HTTP. A local file can be added to the IPFS 
#           filesystem, making it available to the world. Files are identified 
#           by their hashes, so it's caching-friendly. They are distributed 
#           using a BitTorrent-based protocol. (source: Wikipedia)
# Status:  Alpha (Experimental)
# Use Case Possibilities:  
#   1. Easier sharing of data analysis result sets
#   2. Easier access to popular open data sets via a permanent url
#   3. Easier transfer of assets between notebook servers on different infra
# Invoke Command: 
#   %load https://raw.githubusercontent.com/joshisa/jspark/master/jspark/ipfs.py
#
# Author:  Sanjay Joshi  joshisa (at) us(dot)ibm(dot)com
# Author:  Chris Waldon ckwaldon (at) us(dot)ibm(dot)com
# License: Apache 2.0
# Organization:  IBM jStart (http://ibm.com/jstart)

import sys
import path
import os
import subprocess as sub
import signal
import time
from IPython.display import *

# Print Working Directory
prefix = os.getcwd()
proposed = os.path.dirname(os.path.dirname(prefix))
if os.access(proposed, os.W_OK):
    print("Prefix proposal accepted")
    prefix = proposed
elif os.access(prefix, os.W_OK):
    print("Prefix original accepted")
    prefix = prefix
else:
    sys.exit("No writeable directory found")

# Let's setup useful paths
localDir = prefix + "/.local"
shareDir = prefix + "/.local/share"
ipfsDir = shareDir + "/ipfs"
ipfsHomeDir = ipfsDir + "/go-ipfs"
ipfsRepoDir = shareDir + "/ipfsrepo"

# Let's make sure all of the paths are created if they don't exist
!mkdir $localDir 2> /dev/null
!mkdir $shareDir 2> /dev/null
!mkdir $ipfsDir 2> /dev/null
!mkdir $ipfsRepoDir 2> /dev/null

# Let's now define some IMPORTANT env vars
os.environ["PATH"] += os.pathsep + ipfsHomeDir
os.environ["IPFS_PATH"] = ipfsRepoDir
os.environ["IPFS_LOGGING"] = "" #<empty>, info, error, debug

print("prefix = " + prefix)
print("shareDir = " + shareDir)
print("ipfs Dir = " + ipfsDir)
print("IPFS Repo Dir = " + ipfsRepoDir) 

# Define an IPFS Helper Class
class ipfs():
    def __init__(self):
        cmd = ['ipfs', 'version']
        p = sub.Popen(cmd, stdout=sub.PIPE,
                   stderr=sub.PIPE)
        out, err = p.communicate()
        try:
            out = out.decode("utf-8")
            err = err.decode("utf-8")
        except:
            pass
        print(err)
        print(out)
        cmd = ['ipfs', 'init']
        p = sub.Popen(cmd, stdout=sub.PIPE,
                   stderr=sub.PIPE)
        out, err = p.communicate()
        try:
            out = out.decode("utf-8")
            err = err.decode("utf-8")
        except:
            pass
        print(err)
        print(out)
        self.daemonStart()
        
    def setLog(self, loglevel=""):
        os.environ["IPFS_LOGGING"] = loglevel
        
    def daemonStart(self):
        p = sub.Popen("nohup ipfs daemon > nohup.out 2>&1 &", shell=True)
        (result, err) = p.communicate()
        time.sleep(5)
        output = !cat nohup.out
        log = '\n'.join(str(x) for x in output)
        print(log)

    def daemonStop(self):
        PID = !ps -ef | grep "\bipfs daemon\b" | awk '{print $2}'
        os.kill(int(PID[0]), signal.SIGTERM)
        time.sleep(5)
        log = !cat nohup.out
        log = '\n'.join(str(x) for x in output)
        print(log)
    
# Let's test to see if ipfs already exists in this notebook workspace
isIPFSInstalled = os.path.isfile(ipfsHomeDir + "/ipfs")
if isIPFSInstalled:
    print("Congratulations! IPFS is already installed within your notebook user space")
else:
    print("IPFS is NOT installed within this notebook's user space")
    print("Initiating installation sequence ...")
    
    print("    Downloading and Installing the IPFS binary")
    !wget https://dist.ipfs.io/go-ipfs/v0.4.3/go-ipfs_v0.4.3_linux-amd64.tar.gz -O $ipfsDir/go-ipfs_v0.4.3_linux-amd64.tar.gz
    !tar -zxvf $ipfsDir/go-ipfs_v0.4.3_linux-amd64.tar.gz -C $ipfsDir >/dev/null
    # Retest ipfs existence
    isIPFSInstalled = os.path.isfile(ipfsHomeDir + "/ipfs")
    print("Congratulations!! IPFS is now installed within your notebook")
    print("To validate, run the following command within a cell:")
    print("")
    print("        ipfs = ipfs()")


In [ ]:
#!rm -rf /gpfs/global_fs01/sym_shared/YPProdSpark/user/s2d0-c6f68dc1ef4cd0-aba0aba196d4/.local

<span class="label label-success">Success</span><br/><br/>
<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Awesome Sauce!</strong> Let's instantiate our helper class to start the daemon. If you see an error about the daemon already running, that's a good thing. Don't panic.
</div>

In [ ]:
ipfs=ipfs()

<span class="label label-success">Success</span><br/><br/>
<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Alright!</strong> Let's download and import the python bindings.
</div>

In [ ]:
!pip install ipfsapi --user
import ipfsapi
api = ipfsapi.Client("localhost", 5001)

<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Pretty cool, eh?</strong> At this point we have initialized our local repository, started the IPFS daemon and verified that our binary is installed properly.<br/>We should now attempt to pull some content from the IPFS network.  The help message mentions a readme, lets look at that.
</div>

In [ ]:
print api.cat("QmYwAPJzv5CZsnA625s3Xf2nemtYgPpHdWEz79ojWnPbdG/readme")

<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Sweet!</strong> Something special just occurred.<br/><br/>Our IPFS Go client ...<ol><li>queried the peer-to-peer (P2P) network</li><li>fetched content</li><li>assembled content chunks for the readme document</li>
  <li>printed the file content to stdout via a cat cmd</li></ol>
<br/>All via a unique hashed address that specifically references this content<br/><br/>
<strong style="color:purple">Extra Credit</strong>: Readup more on <a href="https://github.com/jbenet/random-ideas/issues/20" target="_new">Merkle DAG</a><br/>
  Now let's take a peek at what our P2P network peers look like.
</div>

In [ ]:
api.swarm_peers()

<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Neato!</strong> This is a listing of some peer nodes that are closest to our node.<br/>
  Hmmmmm, let's inspect our node's details a little bit closer.
</div>

In [ ]:
api.id()

<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Holy Hash Keys, Batman!</strong> The address configuration is driven via the config file listed within the IPFS repo found @ [....]/.local/share/ipfsrepo<br>
</div>

<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Rock and Roll!</strong><br/>
  Let's grab an image and show it. It just so happens that the entire XKCD archive is available over IPFS. Let's get one!
</div>

In [ ]:
api.get("QmSeYATNaa2fSR3eMqRD8uXwujVLT2JU9wQvSjCd1Rf8pZ/1566 - Board Game/1566 - Board Game.png")
Image("1566 - Board Game.png")

<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Try your own comic!</strong><br/>
  The index is <a href="https://gateway.ipfs.io/ipfs/QmSeYATNaa2fSR3eMqRD8uXwujVLT2JU9wQvSjCd1Rf8pZ" title="IPFS XKCD Index">here</a>. Pick one and display it using the code below as a template! Find an image in the index that you like, then copy it's URL from the hash onward. Make sure to substitute '%20' as a space!
</div>

In [ ]:
# Substitute your comic's number and name!
api.get("QmSeYATNaa2fSR3eMqRD8uXwujVLT2JU9wQvSjCd1Rf8pZ/comicnumber - comic name/comicnumber - comic name.png")
Image("comicnumber - comic name.png")

<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>Let's load another image!</strong><br/>
</div>

In [ ]:
api.get("QmVfFbASG11MjUFuvfAJmjKpxRGYstCAw9GmZEpbA7KE7A")
!mv QmVfFbASG11MjUFuvfAJmjKpxRGYstCAw9GmZEpbA7KE7A QmVfFbASG11MjUFuvfAJmjKpxRGYstCAw9GmZEpbA7KE7A.jpg
Image("QmVfFbASG11MjUFuvfAJmjKpxRGYstCAw9GmZEpbA7KE7A.jpg")

<span class="label label-success">Success</span><br/><br/>
<div class="alert alert-info alert-dismissible fade in" role="alert">
  <button type="button" class="close" data-dismiss="alert" aria-label="Close">
    <span aria-hidden="true">&times;</span>
  </button>
  <strong>IBM jStart!</strong> In addition to consumption, IPFS offers you the control to decide if and what content you'd like to provide by your node. Per this <a href="https://github.com/ipfs/examples/tree/master/examples/pinning" target="_new">readme</a> on pinning, <br/>
  <blockquote>Pinning is the mechanism that allows you to tell ipfs to always keep a given object local. ipfs has a fairly aggressive caching mechanism that will keep an object local for a short time after you perform any ipfs operation on it, but these objects may get garbage collected fairly regularly. To prevent that garbage collection simply pin the hash you care about. <span style="color:purple">Objects added through ipfs add are pinned recursively by default.</span></blockquote>
  Let's be good IPFS netizens and add our fetched content (recursive pin) to our node to help facilitate further propagation of this content.<br/><br/>Per the <a href="https://ipfs.io/docs/commands/#ipfs-add" target="_new">IPFS command doc</a>, 
  <blockquote><strong>IPFS ADD</strong>: Adds contents of &lt;path&gt; to ipfs. Use -r to add directories.
    Note that directories are added recursively, to form the ipfs MerkleDAG.</blockquote>
  <strong style="color:purple">Extra Credit</strong>: Check out <a href="https://github.com/ipfs/go-ipfs/issues/590" target="_new"> Github Issue #590</a> to gain a better understanding of Pinning Semantics.
</div>

In [ ]:
api.add("QmVfFbASG11MjUFuvfAJmjKpxRGYstCAw9GmZEpbA7KE7A.jpg")

<div class="progress" style="height: 40px;">
  <div class="progress-bar progress-bar-success progress-bar-striped active" role="progressbar" aria-valuenow="100" aria-valuemin="0" aria-valuemax="100" style="width: 100%;height:40px;">
    <span style="font-size:20px;line-height: 40px;">Congratulations. Your InterPlanetary Tour on Jupyter is now over.  Hope you enjoyed the show!</span>
  </div>
</div>


<div class="jumbotron" style="color:#F9F9FB;background-color:#1a242d !important;">
  <span style="float: right;width:5%;height:5%">
      <img src="https://ipfs.io/ipfs/QmVfFbASG11MjUFuvfAJmjKpxRGYstCAw9GmZEpbA7KE7A"/>
  </span>
  <span style="float: right;width:5%;height:5%">
      <img style="background-color:white" src="https://rawgit.com/jupyter/design/master/logo/svg/jupyter-sq-text.svg" />
  </span>
  <h1 class="display-3">InterPlanetary File System (IPFS) on Jupyter</h1>
  <p class="lead">Jovians and Bluemix users rejoice!</p>
  <hr class="m-y-2">
  <p>
    <img src="https://cdn.hackaday.io/images/5211791461026701572.png" style="margin:0;padding:0;float:right;"/>
  <h2 style="color:orange">Food For Thought</h2>
  <ul><li>IPFS is still very young.  Alpha means YMMV, but the project is open and evolving.</li>
  <li>Data Scientists need data.  Can IPFS provide a more frictionless experience to accessing data?</li>
  <li>Certain JDBC drivers (e.g. <a href="https://github.com/xerial/sqlite-jdbc" target="_new">Xerial's SQLite</a>, ...) <strong>assume/require</strong> database files on local paths.  Can IPFS help data scientists work with these constrained drivers, either through download or via FUSE mount?</li>
  <li>Can access to the IBM Analytics Exchange be streamlined by having the platform generate and provide permanent IPFS references?</li>
  <li>How does IPFS decentralized technology compare to other similar open projects such as <a href="http://dat-data.com/" target="_new">DAT</a>, <a href="http://maidsafe.net/" target="_new">MaidSafe</a> and <a href="http://storj.io/" target="_new">Storj.io</a>?</li>
  <li>IPFS is a technology focused on distribution, not storage, of content.  It is a peer-to-peer hypermedia protocol.</li>
  <li>Can this be an easier way of sharing popular code snippets? Imagine running <span style="color:orange">cat</span> on a url and receiving a python code segment for using the Spark Technology Center's <a href="http://www.spark.tc/stocator-the-fast-lane-connecting-object-stores-to-spark/" target="_new">Stocator library</a> in your data analytics projects.  Rapid access to code examples.  Maintaining a list of best of breed approaches via a list of links sounds convenient.  Maybe this opens up opportunities for a trusted advisor chat bot that I populate with content link lists that I trust?</li>
  <li>What use cases does IPFS make better?  In what cases is P2P unwise or unnecessary?</li>
  <li>How can IBM Cloud offerings differentiate themselves by embracing the momentum around distributed web and mesh networks?</li>
  <li>Did you notice that some of these embedded images are being served via an IPFS resource url hosted @ ipfs.io?</li>
  <li>Opportunities with real-time analytics, Spark streaming and the streaming capabilities of IPFS streams?</li>
  <li>Opportunities for triggering "serverless" lambda functions (e.g. <a href="openwhisk.org" target="_new">OpenWhisk</a>, AWS Lambda, ...) via an IPFS client to emit events, thus causing parallel content movement programmatically (e.g. spin up lambda, fetch content and put it in a blob store)?  Would this require new flavors of lambda functions -- compute optimized vs. bandwidth optimized?</li>
  <li>What next ? ...</li>
  </ul><br/>
  <h2 style="color:orange">Useful References</h2>
  <ul>
      <li><a href="https://en.wikipedia.org/wiki/InterPlanetary_File_System" target="_new">https://en.wikipedia.org/wiki/InterPlanetary_File_System</a></li>
      <li><a href="http://dietzel.me/2015/08/02/EuroPython-2015-Holger-Krekels-Keynote-about-the-interplanetary-filesystem-Wed-22nd-July-2015/" target="_new">http://dietzel.me/2015/08/02/EuroPython-2015-Holger-Krekels-Keynote-about-the-interplanetary-filesystem-Wed-22nd-July-2015/</a></li>
      <li><a href="http://www.wired.com/2016/06/inventors-internet-trying-build-truly-permanent-web/" target="_new">http://www.wired.com/2016/06/inventors-internet-trying-build-truly-permanent-web/</a></li>
      <li><a href="https://blog.acolyer.org/2015/10/05/ipfs-content-addressed-versioned-p2p-file-system/" target="_new">https://blog.acolyer.org/2015/10/05/ipfs-content-addressed-versioned-p2p-file-system/</a></li>
      <li><a href="https://techcrunch.com/2015/10/04/why-the-internet-needs-ipfs-before-its-too-late/" target="_new">https://techcrunch.com/2015/10/04/why-the-internet-needs-ipfs-before-its-too-late/</a></li>
      <li><a href="https://github.com/ipfs/ipfs#overview" target="_new">https://github.com/ipfs/ipfs#overview</a></li>
      <li><a href="https://ipfs.io/ipfs/QmNhFJjGcMPqpuYfxL62VVB9528NXqDNMFXiqN5bgFYiZ1/its-time-for-the-permanent-web.html" target="_new">https://ipfs.io/ipfs/QmNhFJjGcMPqpuYfxL62VVB9528NXqDNMFXiqN5bgFYiZ1/its-time-for-the-permanent-web.html</a></li>
      <li><a href="https://github.com/ipfs/faq/issues" target="_new">https://github.com/ipfs/faq/issues</a></li>
      <li><a href="https://github.com/ipfs/awesome-ipfs" target="_new">https://github.com/ipfs/awesome-ipfs</a></li>
      <li><a href="https://diginomics.com/interview-kyle-drake-juan-benet-ipfs/" target="_new">https://diginomics.com/interview-kyle-drake-juan-benet-ipfs/</a></li>
      <li><a href="https://github.com/ipfs/apps/issues/29" target="_new">IPFS + Jupyter Notebooks Github Discussion Issue</a>&nbsp;&nbsp;<img style="vertical-align: bottom;display: inline-block;" src="http://icons.iconarchive.com/icons/custom-icon-design/valentine/16/heart-icon.png"/></li>
      <li><a href="http://filecoin.io/" target="_new">http://filecoin.io/</a>&nbsp;&nbsp;<img style="vertical-align: bottom;display: inline-block;" src="http://icons.iconarchive.com/icons/custom-icon-design/valentine/16/heart-icon.png"/></li>
      <li><a href="http://arxiv.org/pdf/1407.3561.pdf" target="_new">http://arxiv.org/pdf/1407.3561.pdf</a></li>
      <li><a href="https://www.youtube.com/watch?v=8CMxDNuuAiQ" target="_new">IPFS Video Intro</a>&nbsp;&nbsp;<img style="vertical-align: bottom;display: inline-block;width:2%" src="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2015/03/YouTube-social-circle_red_128px.png"/></li>
      <li><a href="https://db.erisindustries.com/distributed%20business/2015/11/01/eris-and-ipfs/" target="_new">https://db.erisindustries.com/distributed%20business/2015/11/01/eris-and-ipfs/</a></li>
  </ul>
  <h2 style="color:orange">Useful Tools</h2>
  <ul>
      <li><a href="https://github.com/dylanPowers/ipfs-chrome-extension" target="_new">IPFS Chrome Extension</a></li>
      <li><a href="https://github.com/dylanPowers/ipfs-linux-service" target="_new">IPFS Linux Service</a></li>
  </ul>
  </p>
  <p class="lead">
      <div class="text-center">
      <img style="width:10%;height:10%;display:inline" src="https://ipfs.io/ipfs/QmTgtbb4LckHaXh1YhpNcBu48cFY8zgT1Lh49q7q7ksf3M/raster-generated/ipfs-logo-text-128-white.png">&nbsp;&nbsp;
      <a class="btn btn-primary btn-lg" target="_new" href="https://ipfs.io" role="button">Learn more</a>
      <a class="btn btn-primary btn-lg" target="_new" href="http://www-01.ibm.com/software/ebusiness/jstart/contact.shtml" role="button">Contact jStart - Have a conversation with Us</a></div>
  </p>
  
</div>